In [1]:
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen
from datetime import datetime

In [263]:
# Get denormalized-flights data
def get_denorm_data(from_param, to_param) :
    denorm_url = 'http://sfuelepcn1.thaiairways.co.th:3001/denormalized-flights?'
    skip = 'skip=0'
    limit = 'limit=0'
    parameter = denorm_url + from_param +'&' + to_param + '&' + skip + '&' + limit
    return pd.read_json(parameter)

# User input eOFP (Set Limit = 1000, default = 50)
def get_oefp_user_input(skip = '0', limit = '500') :
    skip = str(skip)
    limit = str(limit)

    url = "https://tgeofp.rtsp.us/api/v1/userinputs?skip="+skip+"&limit="+limit
    return pd.read_json(url)

# Get all OFP data (Set limit = 1000, default = 150)
def get_ofp() :
    url = "https://tgeofp.rtsp.us/api/v1/ofp?limit=1000"
    return pd.read_json(url)

# Get JSON of ofp data by specific flightplan id
def get_ofp_by_flightplan(flightplan) :
    try:
        url = "https://tgeofp.rtsp.us/api/v1/ofp/" + flightplan
        response = urlopen(url)
        data = json.loads(response.read())
        return data
    except Exception as e:
        return None

def create_fuelreport_df(denorm_df):
    fuel_report_df = denorm_df[denorm_df.fuelreport.notna()]['fuelreport'].apply(pd.Series)
    fuel_report_df.drop_duplicates(subset=['dep', 'flight_number', 'flight_date', 'aircraft_registration'], inplace=True)
    return fuel_report_df

def count_fuel_data(fuel_report_df):
    """Return DataFram contain number of Fuel report data each month"""
    
    fuel_summary_df = fuel_report_df.sort_values(['flight_date', 'flight_number']).reset_index()[['flight_date','flight_number','aircraft_registration','dep']]
    month = []
    year = []
    for date in fuel_summary_df.flight_date:
        date_time = pd.to_datetime(date)
        month.append(date_time.month)
        year.append(date_time.year)

    fuel_summary_df['month'] = month
    fuel_summary_df['year'] = year
    return fuel_summary_df.groupby(['year','month'])['flight_number'].count()
def merge_flightPlan_eofp(eofp) :

    ## Initialize dataframe with eOFP
    df = eofp

    ## Get OFP data to create joint column on denorm
    for index, row in df.iterrows() :
        flightplan = df.iloc[index]["flightPlan"]
        
        # Get ofp flight information
        ofp_json = get_ofp_by_flightplan(flightplan)

        # Drop Unmatch FlightPlan ID
        if ofp_json == None : 
            df.drop(index = index, axis = 0)
            continue

        dep = ofp_json["flight_key"]['departure_aerodrome']['value']
        arr = ofp_json["flight_key"]['arrival_aerodrome']['value']
        flt_no = "THA" + ofp_json["flight_key"]["flight_number"]
        flt_date = datetime.strptime(ofp_json["flight_key"]["flight_date"],"%Y-%m-%dZ")
        flt_date = flt_date.strftime("%Y-%m-%dT%H:%M:%S.000Z")
        ac_reg = ofp_json["aircraft"]["aircraft_registration"]
        aircraft_registration = ac_reg[:2] + "-" + ac_reg[2:]
        imported_time = datetime.strptime(ofp_json["imported_time"], "%Y-%m-%dT%H:%M:%S.%fZ")
        #imported_time = imported_time.strftime("%Y-%m-%dT%H:%M:%S.000Z") ## TO String

        # Insert new column
        df.loc[index, "departure_aerodrome_icao_code"] = dep
        df.loc[index, "arrival_aerodrome_icao_code"] = arr
        df.loc[index, "flight_number"] = flt_no
        df.loc[index, "flight_date"] = flt_date
        df.loc[index, "aircraft_registration"] = aircraft_registration
        df.loc[index, "ofp_imported_time"] = imported_time
    
    ## Trim only eOFP data with new inserted column
    data_list = [
        "flightPlan","userInput","plannedCheckPoint","createdAt","updatedAt","ofp_imported_time",
        "departure_aerodrome_icao_code", "arrival_aerodrome_icao_code",
        "aircraft_registration", "flight_date", "flight_number",
        ]
    df = df[data_list]
    return df

# def create_merge_df():

In [21]:
# Adjust to/from date
from_param = 'from=2022-04-01T00:00:00.000Z'
to_param = 'to=2022-05-31T23:59:59.000Z'

# Create DataFrame
denorm_df = get_denorm_data(from_param, to_param)
fuel_report_df = create_fuelreport_df(denorm_df)

In [269]:
# Select columns for expand data in each columns
schedule_team_columns = [
    # "ramp_fuel",
    # "ramp_fuel_time",
    # "extra_fuel_time",
    # "departure_gate",
    # "departure_slot",
    # "arrival_gate",
    # "offblock_time",
    # "offblock_fuel",
    # "departure_atis",
    # "departure_atis2",
    # "departure_atis3",
    # "takeoff_runway_1",
    # "flex_temp_1",
    # "vone_1",
    # "vr_1",
    # "vtwo_1",
    # "vref_1",
    # "takeoff_flap_1",
    # "mfra_1",
    # "takeoff_runway_2",
    # "flex_temp_2",
    # "vone_2",
    # "vr_2",
    # "vtwo_2",
    # "vref_2",
    # "takeoff_flap_2",
    # "mfra_2",
    # "takeoff_runway_3",
    # "flex_temp_3",
    # "vone_3",
    # "vr_3",
    # "vtwo_3",
    # "vref_3",
    # "takeoff_flap_3",
    # "mfra_3",
    # "actual_zfw",
    # "actual_tof",
    # "actual_tow",
    # "actual_trip_fuel",
    # "actual_ldw",
    # "dli",
    # "lizfw",
    # "zfwcg",
    # "maczfw",
    # "takeoff_alternate",
    # "eet",
    # "cockpit_crew",
    # "cabin_crew",
    # "pax_a",
    # "pax_b",
    # "pax_c",
    # "pax_d",
    # "pax_e",
    # "sob",
    # "infant",
    # "clearance",
    # "pfd1",
    # "pfd2",
    # "stby",
    # "fuel_onboard",
    # "fuel_uplift",
    # "fuel_before_uplift",
    # "density",
    # "supplier",
    # "airborne_time",
    # "airborne_fuel",
    # "toc_fl",
    # "toc_fuel",
    # "last_wpt_id",
    # "estimate_landingFuel_at_toc",
    # "dest_atis",
    # "alternate_atis",
    # "estimate_landingFuel_at_tod",
    # "tod_fuel",
    "landing_time",
    # "landing_fuel",
    "flight_time",
    "onblock_time",
    "onblock_fuel",
    # "actual_burn_fuel",
    "block_time",
    "pf",
    "pm",
    "pc1",
    "pc2",
    # "water_uplift",
    # "water_remain",
    # "apu_flight_level",
    # "apu_oat",
    # "apu_remark",
    # "apu_start_satisfactory",
    # "apu_shutdown_afterstart",
    # "autoland_performed",
    # "autoland_satisfactory",
    # "lowvis_simulated",
    # "est_zfw",
    # "taxi_fuel",
    # "xtra_reason_other",
    # "xtra_reason_weather",
    # "xtra_reason_airtraffic",
    # "xtra_reason_notam",
    # "xtra_reason_actechnical",
    # "xtra_reason_otp",
    # "nadp_1",
    # "nadp_2",
    # "nadp_3",
    # "takeoff_cg",
    # "cat23_weather",
    # "takeoff_cg_1",
    # "takeoff_cg_2",
    # "takeoff_cg_3",
]
userInput_columns_selected =[
    "ramp_fuel",
    # "ramp_fuel_time",
    # "extra_fuel_time",
    # "departure_gate",
    # "departure_slot",
    # "arrival_gate",
    "offblock_time",
    "offblock_fuel",
    # "departure_atis",
    # "departure_atis2",
    # "departure_atis3",
    # "takeoff_runway_1",
    # "flex_temp_1",
    # "vone_1",
    # "vr_1",
    # "vtwo_1",
    # "vref_1",
    # "takeoff_flap_1",
    # "mfra_1",
    # "takeoff_runway_2",
    # "flex_temp_2",
    # "vone_2",
    # "vr_2",
    # "vtwo_2",
    # "vref_2",
    # "takeoff_flap_2",
    # "mfra_2",
    # "takeoff_runway_3",
    # "flex_temp_3",
    # "vone_3",
    # "vr_3",
    # "vtwo_3",
    # "vref_3",
    # "takeoff_flap_3",
    # "mfra_3",
    "actual_zfw",
    # "actual_tof",
    # "actual_tow",
    "actual_trip_fuel",
    # "actual_ldw",
    # "dli",
    # "lizfw",
    # "zfwcg",
    # "maczfw",
    # "takeoff_alternate",
    # "eet",
    # "cockpit_crew",
    # "cabin_crew",
    # "pax_a",
    # "pax_b",
    # "pax_c",
    # "pax_d",
    # "pax_e",
    # "sob",
    # "infant",
    # "clearance",
    # "pfd1",
    # "pfd2",
    # "stby",
    # "fuel_onboard",
    # "fuel_uplift",
    # "fuel_before_uplift",
    # "density",
    # "supplier",
    "airborne_time",
    "airborne_fuel",
    # "toc_fl",
    "toc_fuel",
    # "last_wpt_id",
    "estimate_landingFuel_at_toc",
    # "dest_atis",
    # "alternate_atis",
    "estimate_landingFuel_at_tod",
    "tod_fuel",
    "landing_time",
    "landing_fuel",
    "flight_time",
    "onblock_time",
    "onblock_fuel",
    "actual_burn_fuel",
    "block_time",
    "pf",
    "pm",
    "pc1",
    "pc2",
    # "water_uplift",
    # "water_remain",
    # "apu_flight_level",
    # "apu_oat",
    # "apu_remark",
    # "apu_start_satisfactory",
    # "apu_shutdown_afterstart",
    # "autoland_performed",
    # "autoland_satisfactory",
    # "lowvis_simulated",
    "est_zfw",
    # "taxi_fuel",
    # "xtra_reason_other",
    # "xtra_reason_weather",
    # "xtra_reason_airtraffic",
    # "xtra_reason_notam",
    # "xtra_reason_actechnical",
    # "xtra_reason_otp",
    # "nadp_1",
    # "nadp_2",
    # "nadp_3",
    # "takeoff_cg",
    # "cat23_weather",
    # "takeoff_cg_1",
    # "takeoff_cg_2",
    # "takeoff_cg_3",
]

fuel_report_col=[
    'actual_takeoff_fuel',
    'actual_takeoff_time',
    'aircraft_registration',
    'aircraft_type',
    'arr',
    'company_fuel',
    'flight_date',
    'dep',
    'esad',
    'extra_fuel',
    'flight_number',
    'actual_flt_time',
    "plan_flt_time",
    "actual_burn",
    "plan_burn",
    "actual_landing_fuel",
    "plan_landing_fuel",
    "actual_landing_time",
    "actual_off_block_time",
    "actual_on_block_fuel",
    "plan_on_block_fuel",
    "actual_on_block_time",
    "actual_block_fuel",
    "plan_block_fuel",
    "actual_taxi_in_fuel",
    "actual_taxi_in_time",
    "actual_taxi_fuel",
    "plan_taxi_fuel",
    "actual_taxi_out_time",
    "plan_taxi_out_time",
    "actual_zfw",
    "plan_zfw",
]

qar_col=[
    # 'takeoff_position',
    # 'takeoff_wind',
    # 'landing_position',
    # 'landing_wind',
    # 'touch_down_attitude',
    # 'actual_time_arrival',
    'actual_flight_time',
    # 'apu_used_during_taxi_in',
    # 'assumed_temp',
    'average_cruise_altitude',
    # 'average_n1_reverser_deployed',
    # 'climb_duration',
    # 'climb_fuel_burn',
    # 'cruise_duration',
    # 'cruise_fuel_burn',
    # 'cruise_distance',
    # 'descent_fuel_burn',
    # 'gross_weight',
    # 'landing_config_height',
    # 'landing_flaps',
    # 'max_landing_lateral_g',
    # 'max_landing_vertical_g',
    # 'max_taxi_out_lateral_g',
    # 'max_taxi_out_speed',
    # 'nadp',
    # 'one_engine_taxi_in',
    'overall_fuel_used',
    # 'pack_off_takeoff',
    'ramp_fuel',
    # 'reduced_flaps_landing',
    # 'stabilized_approach_height',
    # 'takeoff_flaps',
    # 'takeoff_oat',
    'taxi_in_duration',
    'taxi_out_fuel',
    # 'thrust_reduction_height',
    # 'top_of_climb_altitude',
    # 'top_of_descent_altitude',
    'zero_fuel_weight',
    # 'auto_land_performed',
    # 'idle_reverse_landing_performed',
    # 'landing_sector_fuel_burn',
    # 'acceleration_height',
    # 'flaps_retraction_height'
]

fuel_df_col = [
    # # Flight plan
    # '0_plan',
    'block_fuel_plan',
    'taxi_fuel_plan',
    'trip_fuel_plan',
    'contingency_fuel',
    'company_fuel',
    'final_reserve_fuel',
    'extra_fuel',
    'additional_fuel',
    'alternate_fuel',
    # # Load sheet
    # '0_order',
    'block_fuel_order',
    'taxi_fuel_order',
    'trip_fuel_order'
]

fuel_report_col =[
     # # Data from fuel acars
    'aircraft_registration',
    'aircraft_type',
    'arr',
    'dep',
    'flight_date',
    'flight_number',
    'actual_block_fuel',
    'actual_burn',
    'actual_flt_time',
    'actual_fob',
    'actual_landing_date',
    'actual_landing_fuel',
    'actual_landing_time',
    'actual_off_block_date',
    'actual_off_block_time',
    'actual_on_block_date',
    'actual_on_block_fuel',
    'actual_on_block_time',
    'actual_takeoff_date',
    'actual_takeoff_fuel',
    'actual_takeoff_time',
    'actual_taxi_fuel',
    'actual_taxi_in_fuel',
    'actual_taxi_in_time',
    'actual_taxi_out_time',
    'actual_zfw',
    'company_fuel',
    'esad',
    # 'eta_date',
    # 'eta_time',
    # 'etd_date',
    # 'etd_time',
    # 'excess_fuel',
    'extra_fuel',
    'plan_block_fuel',
    'plan_burn',
    'plan_flt_time',
    # 'plan_landing_date',
    # 'plan_landing_fuel',
    # 'plan_landing_time',
    # 'plan_off_block_date',
    # 'plan_off_block_time',
    # 'plan_on_block_date',
    'plan_on_block_fuel',
    # 'plan_on_block_time',
    # 'plan_takeoff_date',
    # 'plan_takeoff_time',
    'plan_taxi_fuel',
    # 'plan_taxi_in_fuel',
    # 'plan_taxi_in_time',
    'plan_taxi_out_time',
    'plan_zfw',
    # 'sta_date',
    # 'sta_time',
    'std_date',
    'std_time',
]

In [23]:
# Select column and export to csv file
file_name_date = from_param[5:9] + from_param[10:12]+ from_param[13:15] + "_" + to_param[3:7] + to_param[8:10]+ to_param[11:13]
fuel_report_df.to_csv('fuel_report_data_'+file_name_date+'.csv')
fuel_report_df = fuel_report_df[fuel_report_col]

In [ ]:
# Get latest eOFP data

total_data = len(fuel_report_df)
total_loop = total_data // 500
# total_loop = 4

eofp_df = get_oefp_user_input()

for i in np.arange(total_loop):
    limit = 500
    skip = 500 + 500 * i
    chunk = get_oefp_user_input(skip=skip, limit=limit)
    
    # Don't append last chunk
    if len(chunk) < 500:
        break

    eofp_df = pd.concat([eofp_df, chunk], ignore_index= True)

In [273]:
schedule_team_data = merge_flightPlan_eofp(eofp_df)

In [275]:
back_up = schedule_team_data.copy()

In [279]:
# schedule_team_data = schedule_team_data[['departure_aerodrome_icao_code','arrival_aerodrome_icao_code', 'aircraft_registration', 'flight_date','flight_number','userInput']]
# schedule_team_data = schedule_team_data.join(schedule_team_data.userInput.apply(pd.Series)[schedule_team_columns])
# schedule_team_data = schedule_team_data[schedule_team_data['flight_date'] >= '2022-04-01']
schedule_team_data.drop(columns='userInput', inplace=True)

In [281]:
schedule_team_data.to_csv('schedule_team_by_ken_the_great.csv')

In [247]:
eofp_df[eofp_df['_id'].isin(dup_id.index)]

,_id,flightPlan,udid,__v,createdAt,enrouteweather,plannedCheckPoint,updatedAt,userInput,fixes
2486,6131876fbd239f69ec28f713,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '6131876fbd239f521e28f714', 'eta': '...."
2487,6130487fbd239f12c728e783,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '6130487fbd239f784628e784', 'eta': '1..."
2488,613187c2bd239f98a128f73f,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '613187c2bd239f409028f740', 'eta': '...."
2489,61318882bd239f7fb928f76b,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '61318882bd239fa06728f76c', 'eta': '0..."
2490,6069461f21ce07aab2d8afa6,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '6069461f21ce0785a0d8afa7', 'eta': '0..."
2491,613076b1bd239fa7c728e85f,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '613076b1bd239f119d28e860', 'eta': '...."
2492,613188a0bd239fb6da28f7ba,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '613188a0bd239fde0528f7bb', 'eta': '0..."
2493,61318b72bd239f852428f809,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '61318b72bd239f08ca28f80a', 'eta': '1..."
2494,6130c12abd239fc80028eb45,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '6130c12abd239ff27128eb46', 'eta': '...."
2495,61318cf4bd239fdc2128f84d,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '61318cf4bd239f5fad28f84e', 'eta': '1..."


In [251]:
start = 2490
end = 2500
all_df.iloc[start:end]._id == eofp_df.iloc[start:end]._id

2490    True
2491    True
2492    True
2493    True
2494    True
2495    True
2496    True
2497    True
2498    True
2499    True
Name: _id, dtype: bool

In [198]:
all_df.iloc[2480],

,_id,flightPlan,udid,__v,createdAt,enrouteweather,plannedCheckPoint,updatedAt,userInput,fixes
2546,6131876fbd239f69ec28f713,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '6131876fbd239f521e28f714', 'eta': '...."
2547,613187c2bd239f98a128f73f,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '613187c2bd239f409028f740', 'eta': '...."
2548,61318882bd239f7fb928f76b,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '61318882bd239fa06728f76c', 'eta': '0..."
2549,613188a0bd239fb6da28f7ba,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '613188a0bd239fde0528f7bb', 'eta': '0..."
2550,61318b72bd239f852428f809,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '61318b72bd239f08ca28f80a', 'eta': '1..."
2551,61318cf4bd239fdc2128f84d,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '61318cf4bd239f5fad28f84e', 'eta': '1..."
2552,61556e454ad993d0d6b82d4e,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '61556e454ad993f461b82d4f', 'eta': '0..."
2553,616c260af07f3e584f37ca97,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '616c260af07f3eda4437ca98', 'eta': '...."


In [193]:
all_df.iloc[2483:2499]

,_id,flightPlan,udid,__v,createdAt,enrouteweather,plannedCheckPoint,updatedAt,userInput,fixes
2483,606730182c900853cb70d520,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '606730182c9008620870d521', 'eta': '0..."
2484,6067472a21ce076b6fd89137,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '6067472a21ce0704f0d89138', 'eta': '0..."
2485,6067478421ce076e63d8914b,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '6067478421ce079538d8914c', 'eta': '0..."
2486,6067478521ce07f431d8915f,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '6067478521ce078ae1d89160', 'eta': '0..."
2487,6067483421ce07757bd891db,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '6067483421ce0776f4d891dc', 'eta': '0..."
2488,6067484621ce0706bfd8920e,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '6067484621ce072dbcd8920f', 'eta': '0..."
2489,6067485921ce0708e8d89241,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '6067485921ce077a5ed89242', 'eta': '0..."
2490,6067485f21ce0719f3d89274,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '6067485f21ce0759c2d89275', 'eta': '0..."
2491,6067488b21ce078187d892a7,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '6067488b21ce0724c9d892a8', 'eta': '0..."
2492,6067489e21ce0786a8d892da,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '6067489e21ce0706add892db', 'eta': '1..."


In [144]:
np.sort(ind)

array([483, 484, 485, 486, 488, 489, 490, 491, 492, 493, 496, 497, 498,
       499], dtype=int64)

In [159]:
eofp_df.sort_index()

,_id,flightPlan,udid,__v,createdAt,enrouteweather,plannedCheckPoint,updatedAt,userInput,fixes
0,62985ec500f4d4e72db3503d,6294b8cabc282470baf97e15,None,0,2022-06-02T06:55:01.103Z,[],"[{'_id': '62985f85bc28246693fa4f16', 'eta': '1...",2022-06-02T06:58:13.154Z,"{'departure_gate': '08', 'departure_slot': '-'...",NaN
0,6281c8af81d3483f4cf153b7,627bb47ed1628c71b1923954,NaN,0,2022-05-16T03:44:47.380Z,[],"[{'_id': '6281c8afd1628c81639357d4', 'eta': '1...",2022-05-16T03:44:47.380Z,"{'ramp_fuel': 105500, 'ramp_fuel_time': '1352'...",NaN
0,626e13384921904c3580632c,626d1b639b5250596b34b337,NaN,0,2022-05-01T04:57:28.425Z,[],"[{'_id': '6271c7619b525005f935829c', 'eta': '1...",2022-05-04T00:22:57.384Z,"{'departure_gate': '08', 'departure_slot': Non...",NaN
0,6260196b4921904c35777dfb,625f15efa4ac700133ccfc3c,NaN,0,2022-04-20T14:32:11.924Z,[],"[{'_id': '62601973a4ac70f573cd2cba', 'eta': '0...",2022-04-20T14:32:19.963Z,"{'ramp_fuel': 31800, 'ramp_fuel_time': '0449',...",NaN
0,606e277c75ac2830844d73aa,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '606e277c75ac2876574d73ab', 'eta': '...."
...,...,...,...,...,...,...,...,...,...,...
499,628dc0c800f4d4e72dac5219,628c445636acc6463d8aa664,None,0,2022-05-25T05:38:16.216Z,[],"[{'_id': '628dc0d4720b7438fcb0157f', 'eta': '0...",2022-05-25T05:38:28.812Z,"{'departure_gate': 'C10', 'departure_slot': No...",NaN
499,62601a864921904c35777eb9,625f14c2a4ac70062accfbc5,NaN,0,2022-04-20T14:36:54.838Z,[],"[{'_id': '62601a8ca4ac708b3dcd2d21', 'eta': '0...",2022-04-20T14:37:00.370Z,"{'ramp_fuel': 31511, 'ramp_fuel_time': '0446',...",NaN
499,6281cf8781d3483f4cf157ef,6281664dd1628cabb7933dbe,NaN,0,2022-05-16T04:13:59.068Z,[],"[{'_id': '6281cf87d1628c16f893590c', 'eta': '0...",2022-05-16T04:13:59.068Z,"{'ramp_fuel': 42800, 'ramp_fuel_time': '0940',...",NaN
499,6130c153bd239f13e328ebe3,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,"[{'_id': '6130c153bd239f6a9228ebe4', 'eta': '...."


In [ ]:
e